# Imports and Constants

An API key must be given. Registration required first.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil import parser

from dotenv import load_dotenv

from tqdm.notebook import tqdm

# import metalhistory as mh
import metalhistory.data_query_functions as dqf

import requests
import pandas as pd
import xmltodict
import time

In [3]:
tqdm.pandas()

# Data Preprocessing

In [4]:
%%time
df_csv = pd.read_csv('data/MA_10k_albums.csv')
df_csv

CPU times: user 8.77 ms, sys: 107 µs, total: 8.88 ms
Wall time: 8.47 ms


,artist,album,MA_score
0,Slayer,Reign in Blood,36.01
1,Metallica,Kill 'Em All,33.39
2,Hades Archer,Penis Metal,32.67
3,Iron Maiden,Iron Maiden,32.38
4,Metallica,Master of Puppets,31.83
...,...,...,...
9995,Iron Maiden,Live at the Rainbow,1.92
9996,Jorn,Worldchanger,1.92
9997,Juggernaut,Trouble Within,1.92
9998,Lacrimas Profundere,Memorandum,1.92


# Alternative
Don't use album matches! Because they always crash after a while!

In [10]:
%%time
lastfm = dqf.LastFM()

LIMIT = 1000
result_df = df_csv.head(LIMIT)

fields =['artist', 'name', 'release-date', 'listeners', 'playcount', 'tags', 'mbid', 'url', 'image']

result_df['lastfm_info'] = result_df.progress_apply(lambda row: lastfm.get_album_info(artist=row['artist'], album=row['album'], fields=fields), axis=1)

result_df= result_df.lastfm_info.apply(pd.Series)
fields.append('ignored tags')
result_df.rename(columns={'name':'album'}, inplace=True)
result_df = result_df.set_index(['artist', 'album'])
result_df.drop([0], axis=1)

show_fields =['release-date', 'listeners', 'playcount', 'tags', 'ignored tags']
result_df[show_fields]

  0%|          | 0/1000 [00:00<?, ?it/s]

Response code 503. Waiting for 5 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 2 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 5 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 5 seconds.
Response code 503. Waiting for 5 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 5 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 4 seconds.
Response code 503. Waiting for 3 seconds.
Response code 503. Waiting for 3 s

,,release-date,listeners,playcount,tags,ignored tags
artist,album,,,,,
Slayer,Reign in Blood,1986-10-07,822528,15587243,"[thrash metal, speed metal, heavy metal]","[albums I own, metal]"
Metallica,Kill 'Em All,1983-07-25,582246,14372137,"[thrash metal, heavy metal, speed metal]","[albums I own, metal]"
NaN,NaN,NaN,NaN,NaN,NaN,NaN
Iron Maiden,Iron Maiden,1980-04-14,390345,7084675,"[heavy metal, NWOBHM]","[albums I own, metal, 1980]"
Metallica,Master of Puppets,2003,949182,22464934,"[thrash metal, heavy metal]","[albums I own, metal, favourite albums]"
...,...,...,...,...,...,...
Unexpect,In A Flesh Aquarium,2006-08-22,34838,746878,[],"[Avant-garde Metal, experimental, Progressive ..."
Xasthur,Telepathic With the Deceased,2004-06,24253,387969,"[black metal, doom metal]","[albums I own, depressive black metal, dark]"
Burzum,Umskiptar,2012-05-21,37290,1016213,[black metal],"[2012, atmospheric black metal, Skaldic Metal,..."


In [11]:
result_df[show_fields].sort_values(by='listeners', ascending=False).to_csv('./data/proc_MA_1000_albums_non_cumulative.csv')